# Interpretable AI for Deep Learning Models
###### ARI3205-SEM1-A-2526
### Proffessor: Konstantinos Makantasis
##### Authors: Love Carlander Strandang (2513558), Edvin Sanfridsson (2515058)



## 0. Setup

This section contains:
- Imports + environment info
- Reproducibility (seeds)
- Generic training loops (tabular + vision)
- Metrics + plotting utilities
- Model definitions (Tabular NN + CNN)
- Small wrappers to make PyTorch models compatible with interpretability libraries

In [1]:
import os
import sys
import math
import time
import random
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, Callable, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, f1_score, roc_auc_score, confusion_matrix
)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

import torchvision
import torchvision.transforms as T

print("Python:", sys.version.split()[0])
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("PyTorch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

ModuleNotFoundError: No module named 'numpy'